# Capacitive simulation of qubit decay in charge line

In [ ]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [ ]:
from chip_initialization import create_chip_and_gui
design, gui = create_chip_and_gui()

from design import render_qiskit_metal_design
render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [ ]:
import names as n
import mini_studies as ms
import optimization_targets as ot

In [ ]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)

## Creating design analysis objects

In [ ]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
    )

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + n.CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

## Optimization step

In [ ]:
# number of runs of optimization and number of passes for simulation at each run
group_runs = 10
group_passes_cap = 5

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()